In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# === Загрузка и подготовка данных ===
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
data = np.array(data)
np.random.shuffle(data)

# Dev/test
data_dev = data[:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:] / 255.

# Train
data_train = data[1000:].T
Y_train = data_train[0]
X_train = data_train[1:] / 255.
_, m = X_train.shape

# === Настройка B-сплайнов ===
n_basis = 10
degree = 3
knots = np.linspace(0, 1, n_basis + degree + 1)

def b_spline_basis(x, k, d, knots):
    if d == 0:
        return ((x >= knots[k]) & (x < knots[k + 1])).astype(float)
    denom1 = knots[k + d] - knots[k]
    denom2 = knots[k + d + 1] - knots[k + 1]
    term1 = 0
    if denom1 > 0:
        term1 = (x - knots[k]) / denom1 * b_spline_basis(x, k, d - 1, knots)
    term2 = 0
    if denom2 > 0:
        term2 = (knots[k + d + 1] - x) / denom2 * b_spline_basis(x, k + 1, d - 1, knots)
    return term1 + term2

def compute_spline_features(X):  # X.shape = (784, m)
    d, m = X.shape
    Phi = np.zeros((d * n_basis, m))
    for i in range(d):  # по каждому признаку
        for b in range(n_basis):
            B = b_spline_basis(X[i], b, degree, knots)
            Phi[i * n_basis + b] = B
    return Phi

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/digit-recognizer/train.csv'

In [ ]:
def init_params():
    # Сплайн-слой 1
    S1 = np.random.randn(784, n_basis) * 0.01  # вес на каждый базис для каждого признака
    W1 = np.random.randn(hidden_dim, 784 * n_basis) * 0.01
    b1 = np.zeros((hidden_dim, 1))
    
    # Сплайн-слой 2
    S2 = np.random.randn(hidden_dim, n_basis) * 0.01
    W2 = np.random.randn(10, hidden_dim * n_basis) * 0.01
    b2 = np.zeros((10, 1))
    
    return S1, W1, b1, S2, W2, b2

In [ ]:
def softmax(Z):
    A = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return A / np.sum(A, axis=0, keepdims=True)

def forward_prop(X, S1, W1, b1, S2, W2, b2):
    # === Первый слой: 784 → 784 * n_basis → 128 ===
    B1 = compute_spline_features(X)                # (784 * n_basis, m)
    B1_weighted = B1.copy()
    
    # Применим веса сплайнов первого слоя
    for i in range(784):
        B1[i * n_basis:(i + 1) * n_basis] *= S1[i, :, np.newaxis]
    
    Z1 = W1.dot(B1) + b1
    A1 = np.maximum(Z1, 0)

    # === Второй слой: 128 → 128 * n_basis → 10 ===
    B2 = np.zeros((hidden_dim * n_basis, m))
    for i in range(hidden_dim):
        for b in range(n_basis):
            basis_val = b_spline_basis(A1[i], b, degree, knots)
            B2[i * n_basis + b] = S2[i, b] * basis_val

    Z2 = W2.dot(B2) + b2
    A2 = softmax(Z2)
    return B1, Z1, A1, B2, Z2, A2

In [ ]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    return one_hot_Y.T

def backward_prop(Phi, A, Y, W):
    one_hot_Y = one_hot(Y)
    dZ = A - one_hot_Y           # (10, m)
    dW = (1 / m) * dZ.dot(Phi.T)
    db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
    return dW, db

In [ ]:
def softmax(Z):
    Z_shifted = Z - np.max(Z, axis=0, keepdims=True)
    exp_Z = np.exp(Z_shifted)
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    return one_hot_Y.T

In [ ]:
def compute_loss(A2, Y):
    m = Y.shape[0]
    log_probs = -np.log(A2[Y, np.arange(m)] + 1e-8)
    return np.sum(log_probs) / m

In [ ]:
def update_params(W, b, dW, db, lr=0.1):
    W -= lr * dW
    b -= lr * db
    return W, b

def get_predictions(A):
    return np.argmax(A, axis=0)

def get_accuracy(preds, Y):
    return np.mean(preds == Y)

In [ ]:
W, b = init_params()

for i in range(50):  # эпохи
    Z, A, Phi = forward_prop(W, b, X_train)
    dW, db = backward_prop(Phi, A, Y_train, W)
    W, b = update_params(W, b, dW, db, lr=0.5)

    if i % 5 == 0:
        train_preds = get_predictions(A)
        acc = get_accuracy(train_preds, Y_train)
        print(f"Epoch {i}, Train Accuracy: {acc:.4f}")

In [ ]:
def plot_splines(weights):
    x = np.linspace(0, 1, 200)
    for b in range(n_basis):
        y = b_spline_basis(x, b, degree, knots)
        plt.plot(x, y, label=f"B{b}")
    plt.title("B-spline Basis Functions")
    plt.legend()
    plt.grid(True)
    plt.show()